In [1]:
import numpy as np 
import pandas as pd 


In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')
temp_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df = temp_df.iloc[:10000]
df.shape

(10000, 2)

In [4]:
df.drop_duplicates(inplace=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_18740\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [5]:
df.duplicated().sum()

0

In [6]:
import re
def remove_tags(raw_text): 
    cleaned_text = re.sub(re.compile('<.*?>'),'', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\ACER\AppData\Local\Temp\ipykernel_18740\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [8]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\ACER\AppData\Local\Temp\ipykernel_18740\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [9]:
from nltk.corpus import stopwords
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x: " ".join(x))

C:\Users\ACER\AppData\Local\Temp\ipykernel_18740\4112249004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x: " ".join(x))


In [11]:
import gensim

In [13]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [14]:
story = []
for doc in df['review']: 
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent: 
        story.append(simple_preprocess(sent))

In [15]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [16]:
model.build_vocab(story)

In [17]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875584, 6212140)

In [18]:
len(model.wv.index_to_key)

31845

In [20]:
def document_vector(doc): 
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [22]:
document_vector(df['review'][1])

array([-0.24801405,  0.48822856,  0.11560895, -0.07261202, -0.06726646,
       -0.4313471 , -0.06639218,  0.7252386 , -0.32233393, -0.21326694,
       -0.46896848, -0.51745445, -0.06384256,  0.26527962,  0.3237254 ,
       -0.02159648,  0.12780711, -0.5991361 ,  0.00459059, -0.9502217 ,
        0.3273866 ,  0.3245562 ,  0.3182579 ,  0.04814341, -0.1192041 ,
        0.24205665, -0.21311823, -0.2542161 , -0.4171277 ,  0.01468363,
        0.6226802 ,  0.0848913 ,  0.20431831, -0.2924682 , -0.14045234,
        0.5722624 , -0.01157234, -0.18144313, -0.3778161 , -0.84192663,
        0.22542496, -0.50032115, -0.32676342,  0.18212833,  0.33333614,
       -0.30214438, -0.41555628, -0.10546228,  0.17481211,  0.2711517 ,
        0.24076422, -0.4105315 , -0.34619135, -0.01974453, -0.24850982,
        0.1544131 ,  0.03842105, -0.21026029, -0.49803576, -0.05899635,
       -0.11692439, -0.02341319, -0.21767232,  0.1784662 , -0.3946903 ,
        0.4219772 ,  0.05032628,  0.15622994, -0.32676494,  0.34

In [24]:
from tqdm import tqdm

In [25]:
X = []
for doc in tqdm(df['review'].values): 
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [06:08<00:00, 27.11it/s]


In [27]:
X= np.array(X)

In [28]:
X.shape

(9983, 100)

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['sentiment'])

In [30]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1) 

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [35]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7801702553830746